<a href="https://colab.research.google.com/github/luanps/pyserini/blob/master/Pyserini_LTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LTR - Learning-To-Rank Reranking Models for MS MARCO Passage


### Install Dependencies

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
!gsutil cp gs://luanps/information_retrieval/pyserini/bm25/run.msmarco-passage.bm25tuned.txt .

Copying gs://luanps/information_retrieval/pyserini/bm25/run.msmarco-passage.bm25tuned.txt...
/
Operation completed over 1 objects/126.9 MiB.                                    


In [3]:
!git clone https://github.com/luanps/pyserini.git  --recurse-submodules

Cloning into 'pyserini'...
remote: Enumerating objects: 4369, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4369 (delta 0), reused 0 (delta 0), pack-reused 4366
Receiving objects: 100% (4369/4369), 2.13 MiB | 3.83 MiB/s, done.
Resolving deltas: 100% (3001/3001), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/content/pyserini/tools'...
remote: Enumerating objects: 353, done.        
remote: Counting objects: 100% (110/110), done.        
remote: Compressing objects: 100% (84/84), done.        
remote: Total 353 (delta 54), reused 63 (delta 25), pack-reused 243        
Receiving objects: 100% (353/353), 23.60 MiB | 14.29 MiB/s, done.
Resolving deltas: 100% (138/138), done.
Submodule path 'tools': checked out '22492bec66c55033dedfdab5b04457ee93c8ff43'


In [ ]:
!pip install pyserini 

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

In [ ]:
!pip install faiss-cpu

In [ ]:
!wget https://rgw.cs.uwaterloo.ca/JIMMYLIN-bucket0/pyserini-models/model-ltr-ibm.tar.gz -P collections/msmarco-ltr-passage/
!tar -xzvf collections/msmarco-ltr-passage/model-ltr-ibm.tar.gz -C collections/msmarco-ltr-passage/

!wget https://rgw.cs.uwaterloo.ca/JIMMYLIN-bucket0/pyserini-models/model-ltr-msmarco-passage-mrr-v1.tar.gz -P runs/
!tar -xzvf runs/model-ltr-msmarco-passage-mrr-v1.tar.gz -C runs

In [8]:
#Download MSMARCO dataset

!mkdir collections/msmarco-passage
!wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P collections/msmarco-passage
# Alternative mirror:
# wget https://www.dropbox.com/s/9f54jg2f71ray3b/collectionandqueries.tar.gz -P collections/msmarco-passage
!tar xvfz collections/msmarco-passage/collectionandqueries.tar.gz -C collections/msmarco-passage

--2022-03-03 08:59:40--  https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1057717952 (1009M) [application/gzip]
Saving to: ‘collections/msmarco-passage/collectionandqueries.tar.gz’

collectionandquerie 100%[===================>]   1009M  8.19MB/s    in 3m 10s  

2022-03-03 09:02:51 (5.32 MB/s) - ‘collections/msmarco-passage/collectionandqueries.tar.gz’ saved [1057717952/1057717952]

collection.tsv
qrels.dev.small.tsv
qrels.train.tsv
queries.dev.small.tsv
queries.dev.tsv
queries.eval.small.tsv
queries.eval.tsv
queries.train.tsv


In [9]:
!cp -r pyserini/tools/ .
!cp -r pyserini/scripts .

### Run LTR inference

In [ ]:
!mkdir collections/msmarco-ltr-passage/

!python pyserini/scripts/ltr_msmarco/convert_queries.py \
  --input collections/msmarco-passage/queries.eval.small.tsv \
  --output collections/msmarco-ltr-passage/queries.eval.small.json 

!python pyserini/scripts/ltr_msmarco/convert_queries.py \
  --input collections/msmarco-passage/queries.dev.small.tsv \
  --output collections/msmarco-ltr-passage/queries.dev.small.json

!python pyserini/scripts/ltr_msmarco/convert_queries.py \
  --input collections/msmarco-passage/queries.train.tsv \
  --output collections/msmarco-ltr-passage/queries.train.json

!gsutil cp -r collections/msmarco-ltr-passage/  gs://luanps/information_retrieval/pyserini/ltr/

In [ ]:
#!gsutil cp -r   gs://luanps/information_retrieval/pyserini/ltr/msmarco-ltr-passage/ collections/

In [ ]:
!python -m pyserini.search.lucene.ltr  --input run.msmarco-passage.bm25tuned.txt \
  --input-format tsv \
  --model runs/msmarco-passage-ltr-mrr-v1 \
  --index msmarco-passage-ltr \
  --data passage \
  --ibm-model collections/msmarco-ltr-passage/ibm_model/ \
  --queries collections/msmarco-ltr-passage \
--output run.ltr.msmarco-passage.tsv 

Attempting to initialize pre-built index msmarco-passage-ltr.
index-msmarco-passage-ltr-20210519-e25e33f.tar.gz: 13.1GB [15:14, 15.4MB/s]                
Extracting /root/.cache/pyserini/indexes/index-msmarco-passage-ltr-20210519-e25e33f.tar.gz into /root/.cache/pyserini/indexes/index-msmarco-passage-ltr-20210519-e25e33f.a5de642c268ac1ed5892c069bdc29ae3...
Initializing msmarco-passage-ltr...
Attempting to initialize pre-built index msmarco-passage-ltr.
/root/.cache/pyserini/indexes/index-msmarco-passage-ltr-20210519-e25e33f.a5de642c268ac1ed5892c069bdc29ae3 already exists, skipping download.
Initializing msmarco-passage-ltr...
analyzed contents
text_unlemm text_unlemm
text_bert_tok text_bert_tok
IBM model Load takes 16.11 seconds
IBM model Load takes 30.96 seconds
IBM model Load takes 328.76 seconds
IBM model Load takes 99.07 seconds
---------------------loading dev----------------------------------------
(6974598, 2)
(6980,)
rank    999.226074
rel     999.226074
dtype: float64
        

In [1]:
#MRR Eval
!python -m pyserini.eval.msmarco_passage_eval msmarco-passage-dev-subset run.ltr.msmarco-passage.tsv >> ltr_mrr_eval.txt



/usr/bin/python3: Error while finding module specification for 'pyserini.eval.msmarco_passage_eval' (ModuleNotFoundError: No module named 'pyserini')


In [ ]:
#TREC Eval
!python -m pyserini.eval.convert_msmarco_run_to_trec_run --input run.ltr.msmarco-passage.tsv --output run.ltr.msmarco-passage.trec

!python -m pyserini.eval.trec_eval -c -mrecall.1000 \
                                      -mmap msmarco-passage-dev-subset \
                                      run.ltr.msmarco-passage.trec >> ltr_trec_eval.txt

In [ ]:
!python -m pyserini.eval.convert_msmarco_run_to_trec_run \
   

!python -m pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_qrels \
   --input tools/topics-and-qrels/qrels.msmarco-passage.dev-subset.txt --output collections/msmarco-passage/qrels.dev.small.trec

In [ ]:
$ tools/eval/trec_eval.9.0.4/trec_eval -c -mrecall.1000 -mmap \
   collections/msmarco-passage/qrels.dev.small.trec runs/run.ltr.msmarco-passage.trec
map                   	all	0.2551
recall_1000           	all	0.8573       	